In [79]:
# импортируем модуль
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time


In [80]:
# URL сайта, который нужно парсить
url = 'https://metarankings.ru/best-movies/'

# # Прокси-сервер
# proxies = {
#     #'http': '156.54.213.132:8080',
#     'https': '156.54.213.132:8080'
# }

# Заголовки запроса (опционально)
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Выполнение GET-запроса с использованием прокси
response = requests.get(url, headers=headers)

# Проверка успешности запроса
if response.status_code == 200:
    print("Успешное соединение")
else:
    print(f"Не удалось подключиться, код ошибки: {response.status_code}")


# инициализируем html-код страницы 
soup = BeautifulSoup(response.text, 'html.parser')
# считываем заголовок страницы
links = soup.find_all('a', class_='name')
pages = soup.find_all('a', class_='page-numbers')

max_page = max([int(page.text) for page in pages if page.text.isnumeric()])

film_df = {
    'page_url':[],
    'image_url':[],
    'movie_title':[],
    'description':[],
    }
break_flag = False
for page in range(1,max_page + 1):
    if break_flag:
        df = pd.DataFrame(film_df)
        break

    url = f'https://metarankings.ru/best-movies/page/{page}'
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        print(f"Успешное соединение со списком {page}")
    else:
        print(f"Не удалось подключиться, код ошибки: {response.status_code}")
        df = pd.DataFrame(film_df)
        break

    soup = BeautifulSoup(response.text, 'html.parser')
    links = soup.find_all('a', class_='name')

    # Выводим найденные ссылки и их атрибуты href
    for link in links:
        #time.sleep(2)
        href = link.get('href')
        response = requests.get(href, headers=headers)

        # Проверка успешности запроса
        if response.status_code == 200:
            print("Успешное соединение со страницей фильма")
        else:
            print(f"Не удалось подключиться, код ошибки: {response.status_code}")
            break_flag = True
            break

        film_page = BeautifulSoup(response.text, 'html.parser')

        name = film_page.find('div', class_='post-meta')
        #info = film_page.find_all('div', class_='featured-movie')
        descr = film_page.find('div', class_='description')

        page_link = href
        picture_link = film_page.find('img', class_='post-image')['src']
        
        name_text = name.h1.span.text
        descr_text = descr.div.text

        film_df['page_url'].append(page_link)
        film_df['image_url'].append(picture_link)
        film_df['movie_title'].append(name_text)
        film_df['description'].append(descr_text.strip())
    
    if page % 10 == 0:
        df = pd.DataFrame(film_df)
        df.to_csv(f'films_to_{page}')

df = pd.DataFrame(film_df)

Успешное соединение
Успешное соединение со списком 1
Успешное соединение со страницей фильма
Успешное соединение со страницей фильма
Успешное соединение со страницей фильма
Успешное соединение со страницей фильма
Успешное соединение со страницей фильма
Успешное соединение со страницей фильма
Успешное соединение со страницей фильма
Успешное соединение со страницей фильма
Успешное соединение со страницей фильма
Успешное соединение со страницей фильма
Успешное соединение со страницей фильма
Успешное соединение со страницей фильма
Успешное соединение со страницей фильма
Успешное соединение со страницей фильма
Успешное соединение со страницей фильма
Успешное соединение со страницей фильма
Успешное соединение со страницей фильма
Успешное соединение со страницей фильма
Успешное соединение со страницей фильма
Успешное соединение со страницей фильма
Успешное соединение со страницей фильма
Успешное соединение со страницей фильма
Успешное соединение со страницей фильма
Успешное соединение со стра

In [83]:
#df.to_csv(f'films_final.csv')
df.head(10)

,page_url,image_url,movie_title,description
0,https://metarankings.ru/terminator-2-sudnyj-de...,https://metarankings.ru/images/uploads/Termina...,Терминатор 2: Судный день в 3D,Фильм «Терминатор 2: Судный день» в 3D (2017) ...
1,https://metarankings.ru/gran-budushhego/,https://metarankings.ru/images/uploads/edge-of...,Грань будущего,Фильм «Грань будущего» переносит в недалёкое б...
2,https://metarankings.ru/bezumnyj-maks-doroga-y...,https://metarankings.ru/images/uploads/Mad-Max...,Безумный Макс: Дорога ярости,Фильм «Безумный Макс: Дорога ярости» — это пос...
3,https://metarankings.ru/interstellar/,https://metarankings.ru/images/uploads/2014/11...,Интерстеллар,Фильм «Интерстеллар» повествует о путешествиях...
4,https://metarankings.ru/vyzhivshij-2016/,https://metarankings.ru/images/uploads/The-Rev...,Выживший,Выживший — это новая историческая драма с Леон...
5,https://metarankings.ru/oderzhimost-2014/,https://metarankings.ru/images/uploads/Whiplas...,Одержимость,«Одержимость» рассказывает о парне по имени Эн...
6,https://metarankings.ru/gravity/,https://metarankings.ru/images/uploads/Gravity...,Гравитация,"«Гравитация» — это фильм о выживании, действия..."
7,https://metarankings.ru/kingsman-sekretnaya-sl...,https://metarankings.ru/images/uploads/Kingsma...,Kingsman: Секретная служба,Фильм «Kingsman: Секретная служба» основанный ...
8,https://metarankings.ru/amerikanskoe-chtivo-2023/,https://metarankings.ru/images/uploads/2023/09...,Американское чтиво,Фильм «Американское чтиво» рассказывает истори...
9,https://metarankings.ru/pervyj-mstitel-protivo...,https://metarankings.ru/images/uploads/Captain...,Первый мститель: Противостояние,Фильм «Первый мститель: Противостояние» — Разд...
